In [10]:
%reset -f

import gc
gc.collect()  # 強制 Python 回收記憶體

0

In [11]:
import pandas as pd
import numpy as np

In [12]:
# 讀取總表
summary_file = "./combine/combined_250318.csv"  # 修改為你的總表
df = pd.read_csv(summary_file)

# 目標測試條件
target_turns_values = [5, 10, 15, 20, 25]
target_duty_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  # 目標 Duty
target_frequency = 100  # 目標頻率 (kHz)
target_hdc_values = [300, 600, 900, 1200, 1500, 1800, 2100, 2400]  # Hdc(A/m)
target_Bm_values = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]  # B_max(mT)

# 數值比對的誤差範圍 (tolerance)
tolerance_hdc = 5  # 允許 Hdc 誤差範圍 ±10 A/m
tolerance_Bmax = 1  # 允許 B_max 誤差範圍 ±5 mT

# 建立結果表
result_table = []

# 遍歷所有組合，檢查是否有接近的測試條件
for turns in target_turns_values:
    for duty in target_duty_values:
        for hdc in target_hdc_values:
            row = {
                "N1": turns,
                "Duty": duty,
                "Hdc(A/m)": hdc,
                "Frequency (kHz)": target_frequency
            }

            for Bmax in target_Bm_values:
                # 使用條件篩選測試數據（允許數值誤差）
                condition = ((df["N1"] == turns) & (df["Duty_P"] == duty) &
                             (df["Frequency(kHz)"] == target_frequency) &
                             (np.abs(df["Hdc(A/m)"] - hdc) <= tolerance_hdc)
                             &  # Hdc 誤差範圍
                             (np.abs(df["B_max(mT)"] - Bmax) <= tolerance_Bmax
                              )  # B_max 誤差範圍
                             )

                row[f"B_max(mT)={Bmax}mT"] = "✔" if df[condition].shape[
                    0] > 0 else "✘"

            result_table.append(row)

# 轉換成 DataFrame
result_df = pd.DataFrame(result_table)

# **輸出到 CSV**
output_file = "./test_condition_summary.csv"
result_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ 測試條件統計完成，結果已儲存到 {output_file}")


✅ 測試條件統計完成，結果已儲存到 ./test_condition_summary.csv
